### Step 0: Preliminaries
Necessary package installations and imports.

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu117


### Step 1: Create a simple graph
Create a simple graph with three nodes and two edges

In [5]:
import redis
from redis.commands.graph import Graph
from redis.commands.graph.node import Node
from redis.commands.graph.edge import Edge

connection = redis.Redis('localhost', 6379, 0)
graph = Graph(connection, 'pyg')

jane = Node(label='person', properties={'age':32})
graph.add_node(jane)
john = Node(label='person', properties={'age':38})
graph.add_node(john)
japan = Node(label='country', properties={'name': 'Japan'})
graph.add_node(japan)
edge1 = Edge(jane, 'visited', japan, properties={'purpose': 'pleasure'})
graph.add_edge(edge1)
edge2 = Edge(john, 'visited', japan, properties={'purpose': 'pleasure'})
graph.add_edge(edge2)
graph.commit()




In [18]:
import rg_pyg.connection
import rg_pyg.database

db = rg_pyg.database.Database('localhost', 6379)
conn = rg_pyg.connection.Connection('localhost', 6379, 0)

# Read nodes
person_nodes = conn.execute("MATCH (a:person) RETURN a LIMIT 5").get_as_result_set()
print(person_nodes[0][0])
print(person_nodes[1][0])

# Read edges
edges = conn.execute("MATCH (a:person)-[r]-() RETURN r LIMIT 5").get_as_result_set()
print(edges[0][0])
print(edges[1][0])

# Read paths
edges = conn.execute("MATCH p=(a:person)-[r]-() RETURN p LIMIT 5").get_as_result_set()
print(edges[0][0])
print(edges[1][0])

(:person{age:32})
(:person{age:38})
()-[:visited{purpose:"pleasure"}]->()
()-[:visited{purpose:"pleasure"}]->()
<(0)-[0]->(2)>
<(1)-[1]->(2)>


In [20]:
# Read nodes
person_nodes = conn.execute("MATCH (a:person) RETURN a LIMIT 5").get_as_torch_geometric()


AttributeError: 'QueryResult' object has no attribute 'getColumnNames'

In [7]:

feature_store, graph_store = db.get_torch_geometric_remote_backend()

In [8]:
feature_store['person', 'age', 32]


AttributeError: 'RedisGraphFeatureStore' object has no attribute '_tensor_attr_cls'

In [9]:
graph_store[('person', 'country'), 'coo']

NameError: name 'EdgeLayout' is not defined